In [1]:
%cd /kaggle/working

import os

from hydra import compose, initialize
from omegaconf import OmegaConf

with initialize(version_base=None, config_path="../experiments/032_feature_channel"):
    cfg = compose(
        config_name="config.yaml", overrides=["debug=True"], return_hydra_config=True
    )

/kaggle/working


In [2]:
import warnings
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import polars as pl
import seaborn as sns

warnings.filterwarnings("ignore", "is_categorical_dtype")
warnings.filterwarnings("ignore", "use_inf_as_na")
pl.Config.set_tbl_cols(-1)
pl.Config.set_tbl_rows(100)

polars.config.Config

In [3]:
df = pl.read_parquet("input/train.parquet", n_rows=100000)

In [4]:
import xarray as xr

grid_path = "/kaggle/working/misc/grid_info/ClimSim_low-res_grid-info.nc"
grid_info = xr.open_dataset(grid_path)
hyam = grid_info["hyam"].to_numpy()
hybm = grid_info["hybm"].to_numpy()
hyai = grid_info["hyai"].to_numpy()
hybi = grid_info["hybi"].to_numpy()

p0 = 1e5
ps = df["state_ps"].to_numpy()
pressures_array = hyam * p0 + hybm[None, :] * ps[:, None]

pressures_array[0]

array([7.83478113e+00, 1.41108318e+01, 2.52923297e+01, 4.49250635e+01,
       7.86346161e+01, 1.34735576e+02, 2.24477729e+02, 3.61643148e+02,
       5.61583643e+02, 8.40325322e+02, 1.21444894e+03, 1.70168280e+03,
       2.32107981e+03, 3.09143463e+03, 4.02775807e+03, 5.13746323e+03,
       6.41892284e+03, 7.86396576e+03, 9.46300920e+03, 1.12091274e+04,
       1.30977804e+04, 1.51221318e+04, 1.72683435e+04, 1.95265236e+04,
       2.18653833e+04, 2.42423874e+04, 2.66410575e+04, 2.90605803e+04,
       3.15131516e+04, 3.40169391e+04, 3.65888174e+04, 3.92401190e+04,
       4.19760536e+04, 4.47970440e+04, 4.76993790e+04, 5.06735901e+04,
       5.37010959e+04, 5.67515928e+04, 5.97841201e+04, 6.27531457e+04,
       6.56180947e+04, 6.83522047e+04, 7.09463085e+04, 7.34056360e+04,
       7.57415383e+04, 7.79627207e+04, 8.00704969e+04, 8.20599900e+04,
       8.39258449e+04, 8.56688208e+04, 8.72996849e+04, 8.88387642e+04,
       9.03119580e+04, 9.17455140e+04, 9.31618329e+04, 9.45774558e+04,
      

## 変化量そのものを確認

In [59]:
h = 40

tmp = df
tmp = tmp.with_columns(
    [
        (pl.col(f"state_t_{h}") + 1200.0 * pl.col(f"ptend_t_{h}")).alias(f"new_t_{h}"),
        (pl.col(f"ptend_q0001_{h}") * 1200.0),
        (pl.col(f"ptend_q0002_{h}") * 1200.0),
        (pl.col(f"ptend_q0003_{h}") * 1200.0),
        (pl.col(f"state_q0002_{h}") + pl.col(f"state_q0003_{h}")).alias(
            f"state_cloud_water_{h}"
        ),
        (pl.col(f"ptend_q0002_{h}") + pl.col(f"ptend_q0003_{h}")).alias(
            f"ptend_cloud_water_{h}"
        ),
        (
            pl.col(f"state_q0003_{h}")
            / (pl.col(f"state_q0002_{h}") + pl.col(f"state_q0003_{h}") + 1e-60)
        ).alias("state_ice_ratio"),
        (
            pl.col(f"ptend_q0003_{h}")
            / (pl.col(f"ptend_q0002_{h}") + pl.col(f"ptend_q0003_{h}") + 1e-60)
        ).alias("ptend_ice_ratio"),
    ]
)

cols = [
    f"state_t_{h}",
    f"new_t_{h}",
    f"state_q0001_{h}",
    f"ptend_q0001_{h}",
    f"state_cloud_water_{h}",
    f"ptend_cloud_water_{h}",
    f"state_q0002_{h}",
    f"ptend_q0002_{h}",
    f"state_q0003_{h}",
    f"ptend_q0003_{h}",
    "state_ice_ratio",
    "ptend_ice_ratio",
]

tmp[cols].head()

state_t_40,new_t_40,state_q0001_40,ptend_q0001_40,state_cloud_water_40,ptend_cloud_water_40,state_q0002_40,ptend_q0002_40,state_q0003_40,ptend_q0003_40,state_ice_ratio,ptend_ice_ratio
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
273.809924,273.781051,0.001845,-0.000002,7.5343e-8,-6.2786e-11,3.5920e-8,-3.5920e-8,3.9423e-8,-3.9423e-8,0.523242,0.523242
270.031305,269.996256,0.001441,0.000002,0.000002,-1.5434e-9,9.1013e-7,-9.1013e-7,9.4192e-7,-9.4192e-7,0.508581,0.508581
277.506678,277.486619,0.003403,0.000014,3.3294e-7,-2.7745e-10,3.3294e-7,-3.3294e-7,0.0,0.0,0.0,-0.0
277.292413,277.26869,0.003594,-0.000013,0.000004,-3.2619e-9,0.000004,-0.000004,0.0,0.0,0.0,-0.0
266.889962,266.853093,0.001652,0.000005,0.000003,-2.4003e-9,0.000001,-0.000001,0.000002,-0.000002,0.60542,0.60542


In [60]:
max_tmp = 273.28
min_tmp = 253.15

In [61]:
tmp[cols].with_columns(
    ((max_tmp - pl.col(f"state_t_{h}")) / (max_tmp - min_tmp)).alias("theory_ratio")
)

state_t_40,new_t_40,state_q0001_40,ptend_q0001_40,state_cloud_water_40,ptend_cloud_water_40,state_q0002_40,ptend_q0002_40,state_q0003_40,ptend_q0003_40,state_ice_ratio,ptend_ice_ratio,theory_ratio
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
273.809924,273.781051,0.001845,-0.000002,7.5343e-8,-6.2786e-11,3.5920e-8,-3.5920e-8,3.9423e-8,-3.9423e-8,0.523242,0.523242,-0.026325
270.031305,269.996256,0.001441,0.000002,0.000002,-1.5434e-9,9.1013e-7,-9.1013e-7,9.4192e-7,-9.4192e-7,0.508581,0.508581,0.161386
277.506678,277.486619,0.003403,0.000014,3.3294e-7,-2.7745e-10,3.3294e-7,-3.3294e-7,0.0,0.0,0.0,-0.0,-0.209969
277.292413,277.26869,0.003594,-0.000013,0.000004,-3.2619e-9,0.000004,-0.000004,0.0,0.0,0.0,-0.0,-0.199325
266.889962,266.853093,0.001652,0.000005,0.000003,-2.4003e-9,0.000001,-0.000001,0.000002,-0.000002,0.60542,0.60542,0.317439
267.151113,267.143804,0.003568,0.000008,0.00004,5.3431e-9,0.000029,0.000003,0.000011,0.000004,0.266555,0.575264,0.304465
275.533635,275.620951,0.006508,-9.5187e-7,0.000021,1.3566e-9,0.000021,0.000002,1.5481e-8,-1.5481e-8,0.000724,-0.00951,-0.111954
275.388406,275.455323,0.006545,0.000033,0.000031,1.4591e-8,0.000031,0.000018,5.9807e-7,-5.9807e-7,0.0192,-0.034159,-0.104739
270.691832,270.800804,0.004939,-0.000048,0.000194,4.1048e-8,0.000171,0.000043,0.000023,0.000006,0.116168,0.124689,0.128573


In [68]:
tmp[cols].filter(
    (pl.col(f"ptend_q0002_{h}") < 0)
    & (pl.col(f"ptend_q0003_{h}") > 0)
    & ((pl.col(f"state_q0002_{h}") + pl.col(f"ptend_q0002_{h}")).abs() > 1e-20)
).head(50).sort("state_t_40")

state_t_40,new_t_40,state_q0001_40,ptend_q0001_40,state_cloud_water_40,ptend_cloud_water_40,state_q0002_40,ptend_q0002_40,state_q0003_40,ptend_q0003_40,state_ice_ratio,ptend_ice_ratio
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
252.73381,252.791539,0.000852,0.000017,0.000019,1.6179e-8,2.6815e-7,-1.4406e-7,0.000019,0.00002,0.985869,1.00742
253.060034,253.056661,0.000891,-0.000002,0.000022,4.8259e-9,1.2435e-7,-1.0009e-7,0.000022,0.000006,0.994338,1.017284
253.159907,253.249637,0.00104,-0.000021,0.000071,8.6355e-9,8.2312e-7,-5.8448e-7,0.000071,0.000011,0.988482,1.056404
253.295862,253.283616,0.000979,0.000004,0.000019,-2.6649e-10,0.000002,-0.000002,0.000017,0.000002,0.894054,-4.983824
253.315254,253.345578,0.000521,-0.000009,0.000021,6.0184e-10,0.000001,-6.0443e-7,0.00002,0.000001,0.934015,1.836927
253.388558,253.384227,0.000911,-0.000002,0.000022,4.6485e-9,4.1630e-7,-1.8321e-7,0.000021,0.000006,0.980973,1.032845
253.51822,253.586236,0.001075,-0.000026,0.000078,3.0690e-9,0.000003,-0.000001,0.000075,0.000005,0.966443,1.284253
253.632671,253.616735,0.000947,0.00001,0.000037,4.9731e-9,0.000001,-1.8881e-7,0.000036,0.000006,0.967185,1.031638
253.651902,253.75793,0.00109,-0.000021,0.000091,5.6766e-9,0.000004,-8.4433e-7,0.000087,0.000008,0.951429,1.12395


## 変化量を足して変化後の値を元に検証

In [67]:
tmelt = 273.15

tmax_fice = tmelt - 10.0  #! max temperature for cloud ice formation
tmin_fice = tmax_fice - 30.0  #! min temperature for cloud ice formation
tmax_fsnow = tmelt  #! max temperature for transition to convective snow
tmin_fsnow = tmelt - 5.0  #! min temperature for transition to convective snow

In [121]:
h = 30

tmp = df
tmp = (
    tmp.with_columns(
        [
            (pl.col(f"state_t_{h}") + 1200.0 * pl.col(f"ptend_t_{h}")).alias(
                f"new_t_{h}"
            ),
            (pl.col(f"state_q0002_{h}") + 1200.0 * pl.col(f"ptend_q0002_{h}")).alias(
                f"new_q0002_{h}"
            ),
            (pl.col(f"state_q0003_{h}") + 1200.0 * pl.col(f"ptend_q0003_{h}")).alias(
                f"new_q0003_{h}"
            ),
        ]
    )
    .with_columns(
        # 絶対値が元の値の絶対値以下なら０埋めする
        [
            pl.when(pl.col(f"new_q0002_{h}").abs() < pl.col(f"ptend_q0002_{h}").abs())
            .then(0)
            .otherwise(pl.col(f"new_q0002_{h}"))
            .alias(f"new_q0002_{h}"),
            pl.when(pl.col(f"new_q0003_{h}").abs() < pl.col(f"ptend_q0003_{h}").abs())
            .then(0)
            .otherwise(pl.col(f"new_q0003_{h}"))
            .alias(f"new_q0003_{h}"),
        ]
    )
    .with_columns(
        [
            (pl.col(f"new_q0002_{h}") + pl.col(f"new_q0003_{h}")).alias(
                f"cloud_water_{h}"
            ),
            (
                pl.col(f"new_q0003_{h}")
                / (pl.col(f"new_q0002_{h}") + pl.col(f"new_q0003_{h}") + 1e-60)
            ).alias("ice_ratio"),
            (
                (
                    (pl.col(f"new_t_{h}") > tmax_fice)
                    & (pl.col(f"new_t_{h}") > tmax_fsnow)
                )
            ).alias("no_ice"),
            (
                (
                    (pl.col(f"new_t_{h}") < tmin_fice)
                    & (pl.col(f"new_t_{h}") < tmin_fsnow)
                )
            ).alias("all_ice"),
        ]
    )
)

cols = [
    "no_ice",
    "all_ice",
    f"ice_ratio",
    f"state_t_{h}",
    f"new_t_{h}",
    f"cloud_water_{h}",
    f"state_q0002_{h}",
    f"ptend_q0002_{h}",
    f"new_q0002_{h}",
    f"state_q0003_{h}",
    f"ptend_q0003_{h}",
    f"new_q0003_{h}",
]

tmp[cols].head()

no_ice,all_ice,ice_ratio,state_t_30,new_t_30,cloud_water_30,state_q0002_30,ptend_q0002_30,new_q0002_30,state_q0003_30,ptend_q0003_30,new_q0003_30
bool,bool,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
false,false,0.0,243.663599,243.637593,0.0,9.0725e-16,-7.5604e-19,0.0,0.0,0.0,0.0
false,false,0.0,238.911192,238.898024,0.0,9.0725e-16,-7.5604e-19,0.0,0.0,0.0,0.0
false,false,0.0,251.639426,251.591888,0.0,3.4508e-10,-2.8757e-13,0.0,0.000008,-6.6868e-9,0.0
false,false,0.0,250.429055,250.381662,0.0,1.3140e-9,-1.0950e-12,0.0,0.000004,-3.7082e-9,0.0
false,false,1.0,236.938842,236.893842,3.1273e-7,8.5367e-14,-7.1139e-17,0.0,0.000008,-6.7685e-9,3.1273e-7


In [122]:
# 話を単純化するために cloud_water が０のやつは取り除いておく
tmp_filter = tmp.filter(pl.col(f"cloud_water_{h}") != 0)[cols]
tmp_filter.head()

no_ice,all_ice,ice_ratio,state_t_30,new_t_30,cloud_water_30,state_q0002_30,ptend_q0002_30,new_q0002_30,state_q0003_30,ptend_q0003_30,new_q0003_30
bool,bool,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
false,false,1.0,236.938842,236.893842,3.1273e-7,8.5367e-14,-7.1139e-17,0.0,0.000008,-6.7685e-9,3.1273e-7
false,false,1.0,240.631083,240.623996,0.000062,6.9637e-9,-5.8031e-12,0.0,0.000062,2.7574e-11,0.000062
false,false,1.0,249.701027,249.756637,0.000061,1.6521e-8,-1.3768e-11,0.0,0.000062,-3.7104e-10,0.000061
false,false,1.0,250.469308,250.585922,0.000115,6.8749e-8,-5.7291e-11,0.0,0.000115,7.2108e-10,0.000115
false,false,1.0,243.296692,243.272579,0.000028,1.9726e-10,-1.6438e-13,0.0,0.000022,4.8938e-9,0.000028


### q2

In [123]:
tmp_filter.filter(pl.col(f"new_q0002_{h}") == 0)[cols].describe()

statistic,no_ice,all_ice,ice_ratio,state_t_30,new_t_30,cloud_water_30,state_q0002_30,ptend_q0002_30,new_q0002_30,state_q0003_30,ptend_q0003_30,new_q0003_30
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",36966.0,36966.0,36966.0,36966.0,36966.0,36966.0,36966.0,36966.0,36966.0,36966.0,36966.0,36966.0
"""null_count""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",0.0,0.457312,1.0,234.805604,234.812722,0.000018,7.0958e-8,-5.9131e-11,0.0,0.000018,5.8918e-10,0.000018
"""std""",null,null,0.0,12.581315,12.601764,0.000034,3.2954e-7,2.7461e-10,0.0,0.00003,7.6986e-9,0.000034
"""min""",0.0,0.0,1.0,209.798248,209.791489,5.1868e-12,0.0,-7.0590e-9,0.0,0.0,-6.0521e-8,5.1868e-12
"""25%""",null,null,1.0,224.178347,224.164327,5.6302e-7,0.0,-2.6108e-13,0.0,8.0563e-7,-7.5684e-10,5.6302e-7
"""50%""",null,null,1.0,235.15789,235.149595,0.000003,2.1585e-16,-1.7955e-19,0.0,0.000004,7.2438e-12,0.000003
"""75%""",null,null,1.0,247.382118,247.397372,0.000019,3.1329e-10,0.0,0.0,0.000019,6.5144e-10,0.000019
"""max""",0.0,1.0,1.0,254.333341,254.314089,0.000362,0.000008,0.0,0.0,0.000263,1.6334e-7,0.000362


In [124]:
tmp_filter.filter(pl.col(f"new_q0002_{h}") > 0)[cols].describe()

statistic,no_ice,all_ice,ice_ratio,state_t_30,new_t_30,cloud_water_30,state_q0002_30,ptend_q0002_30,new_q0002_30,state_q0003_30,ptend_q0003_30,new_q0003_30
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",2669.0,2669.0,2669.0,2669.0,2669.0,2669.0,2669.0,2669.0,2669.0,2669.0,2669.0,2669.0
"""null_count""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",0.0,0.0,0.949576,253.863867,254.078158,0.000127,0.000009,-1.7782e-9,0.000007,0.000109,8.7442e-9,0.00012
"""std""",null,null,0.043149,1.033146,1.02428,0.000073,0.00001,2.5785e-9,0.000008,0.000058,2.0469e-8,0.000068
"""min""",0.0,0.0,0.779197,247.757344,248.77515,2.0630e-7,0.0,-1.7245e-8,1.5282e-10,5.9091e-8,-1.0136e-7,2.0323e-7
"""25%""",null,null,0.92194,253.190186,253.36684,0.000066,0.000002,-2.7038e-9,8.5103e-7,0.000061,-2.7243e-9,0.000063
"""50%""",null,null,0.958393,253.839074,254.02089,0.000123,0.000005,-1.1156e-9,0.000004,0.000109,5.3241e-9,0.000116
"""75%""",null,null,0.98752,254.529594,254.734821,0.000182,0.000014,-1.9185e-10,0.000011,0.000154,1.8790e-8,0.000171
"""max""",0.0,0.0,0.999994,256.758412,257.171022,0.000374,0.000069,1.1732e-8,0.000055,0.000305,1.4292e-7,0.000349


### q3

In [125]:
tmp_filter.filter(pl.col(f"new_q0003_{h}") > 0)[cols].describe()

statistic,no_ice,all_ice,ice_ratio,state_t_30,new_t_30,cloud_water_30,state_q0002_30,ptend_q0002_30,new_q0002_30,state_q0003_30,ptend_q0003_30,new_q0003_30
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",39635.0,39635.0,39635.0,39635.0,39635.0,39635.0,39635.0,39635.0,39635.0,39635.0,39635.0,39635.0
"""null_count""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",0.0,0.426517,0.996604,236.088977,236.110047,0.000026,6.9431e-7,-1.7489e-10,4.8444e-7,0.000024,1.1383e-9,0.000025
"""std""",null,null,0.016883,13.058114,13.095498,0.000047,0.000003,8.3874e-10,0.000003,0.00004,9.3626e-9,0.000045
"""min""",0.0,0.0,0.779197,209.798248,209.791489,5.1868e-12,0.0,-1.7245e-8,0.0,0.0,-1.0136e-7,5.1868e-12
"""25%""",null,null,1.0,224.989419,224.971749,6.5422e-7,0.0,-2.1968e-12,0.0,9.2770e-7,-7.9244e-10,6.5422e-7
"""50%""",null,null,1.0,236.916377,236.896772,0.000004,4.3608e-15,-1.4236e-18,0.0,0.000005,1.3196e-11,0.000004
"""75%""",null,null,1.0,249.166501,249.197529,0.000027,4.4245e-9,0.0,0.0,0.000027,9.0975e-10,0.000027
"""max""",0.0,1.0,1.0,256.758412,257.171022,0.000374,0.000069,1.1732e-8,0.000055,0.000305,1.6334e-7,0.000362


In [126]:
tmp_filter.filter(pl.col(f"new_q0003_{h}") == 0)[cols].describe()

statistic,no_ice,all_ice,ice_ratio,state_t_30,new_t_30,cloud_water_30,state_q0002_30,ptend_q0002_30,new_q0002_30,state_q0003_30,ptend_q0003_30,new_q0003_30
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""null_count""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",null,null,null,null,null,null,null,null,null,null,null,null
"""std""",null,null,null,null,null,null,null,null,null,null,null,null
"""min""",null,null,null,null,null,null,null,null,null,null,null,null
"""25%""",null,null,null,null,null,null,null,null,null,null,null,null
"""50%""",null,null,null,null,null,null,null,null,null,null,null,null
"""75%""",null,null,null,null,null,null,null,null,null,null,null,null
"""max""",null,null,null,null,null,null,null,null,null,null,null,null


In [119]:
max_tmp = 273.28
min_tmp = 253.15

tmp_filter.with_columns(
    ((max_tmp - pl.col("new_t_45")) / (max_tmp - min_tmp)).alias("theory_ratio")
).with_columns((pl.col("new_t_45")).cast(pl.Int32).alias("t")).group_by("t").agg(
    pl.col("ice_ratio").mean(),
    pl.col("ice_ratio").std().alias("std"),
    pl.col("theory_ratio").mean().alias("theory_ratio"),
).sort(
    by=["t"]
).filter(
    (pl.col("ice_ratio") < 1.0) & (pl.col("ice_ratio") > 0)
)

t,ice_ratio,std,theory_ratio
i32,f64,f64,f64
252,0.999999,0.000006,1.031646
253,0.982964,0.014862,0.982343
254,0.93377,0.015099,0.933432
255,0.88805,0.016489,0.8857
256,0.833731,0.019614,0.83288
257,0.784459,0.016209,0.785793
258,0.733061,0.016929,0.734448
259,0.680779,0.017087,0.683872
260,0.632983,0.016232,0.635739
